In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

In [17]:
path = os.path.abspath(r'C:\Users\gabri\Workplace\LabEstat\code\data\t')
list_archive = os.listdir(path)
list_frames = [0 for x in list_archive]
for index, archive in enumerate(list_archive):
    try:
        list_frames[index] = pd.read_csv(os.path.join(path,archive))
    except Exception:
        list_frames[index] = pd.read_csv(os.path.join(path,archive), encoding='latin1')


In [18]:
exl_ls = [0 for x in list_frames]
for index, i in enumerate(list_frames):
    a = list_frames[index].iloc[:,0]
    exl_ls[index] = list(list_frames[index][a.str.contains('Instituto')].index)
exl_ls

[[0, 9, 14, 18, 32, 34, 42, 51, 54, 64, 69, 79],
 [0, 4, 12, 16, 24, 35, 37, 48, 50, 55, 60, 69],
 [0, 5, 13, 16, 23, 32, 34, 42, 44, 51, 56, 64],
 [0, 5, 13, 16, 23, 33, 41, 43, 50, 55, 63],
 [0, 6, 16, 19, 27, 40, 42, 50, 52, 66, 71, 80],
 [0, 2, 8, 17, 20, 28, 40, 42, 56, 58, 68, 73, 83],
 [],
 [0, 2, 8, 17, 20, 27, 39, 41, 50, 53, 66, 71, 80],
 []]

In [19]:
year = 2014
dic={}
for index, i in enumerate(list_frames):
    try:
        list_frames[index] = i.drop(exl_ls[index])
    except KeyError:
        pass
    dic[year]= pd.Series([''.join(list(x)) for x in list_frames[index].iloc[:,0].str.extract(r'([A-Za-zçãâáéíâêôóúõ\s]+)').values])
    year+=1

In [20]:
cursos = []
for year in dic.keys():
    for curso in list(dic[year].unique()):
        if curso.strip() in cursos:
            continue
        else:
            cursos.append(curso.strip())

In [21]:
cursos

['Artes visuais',
 'Cinema e Audiovisual',
 'Dança',
 'Educação Artística',
 'Museologia',
 'Música',
 'Teatro',
 'Biomedicina',
 'Biotecnologia',
 'Ciências Biológicas',
 'Educação Física',
 'Pedagogia',
 'Ciência da Computação',
 'Ciências Naturais',
 'Estatística',
 'Física',
 'Matemática',
 'Química',
 'Química Industrial',
 'Sistemas de Informação',
 'Direito',
 'Enfermagem',
 'Farmácia',
 'Fisioterapia',
 'Medicina',
 'Nutrição',
 'Odontologia',
 'Terapia ocupacional',
 'Administração',
 'Arquivologia',
 'Arquitetura e urbanismo',
 'Biblioteconomia',
 'Ciências Contábeis',
 'Ciências Econômicas',
 'Serviço Social',
 'Turismo',
 'Licenciatura Integrada em Educação em Ciências Matemática e Linguagens',
 'Ciências Sociais',
 'Filosofia',
 'Geografia',
 'Historia',
 'Intervalar Psicologia',
 'Psicologia',
 'Geofísica',
 'Geologia',
 'Meteorologia',
 'Oceanografia',
 'Comunicação Social',
 'Letras',
 'Engenharia Civil',
 'Engenharia da Computação',
 'Engenharia de Alimentos',
 'Engenh

In [22]:
for t in range(8):
    print(t,'=>',len(list_frames[t].iloc[:,:][list_frames[t].iloc[:,-1].isna()].values),'-',end=' ')
    print(list_frames[t].iloc[:,:][list_frames[t].iloc[:,-1].isna()].values)

0 => 0 - []
1 => 0 - []
2 => 0 - []
3 => 0 - []
4 => 0 - []
5 => 0 - []
6 => 0 - []
7 => 0 - []


In [23]:
dic_cursos = {}
for curso in cursos:
    can=[0 for x in range(8)]
    # print(curso)
    for t in range(8):
        temp = sum([int(x[-1]) for x in list_frames[t][list_frames[t].iloc[:,0].str.contains(curso)].values])
        # print(temp,end=' ')
        can[t]=temp
    # print('-----------------------------------')
    dic_cursos[curso]=can
df_cursos = pd.DataFrame(dic_cursos, index=[x for x in range(2014,2022)])
df_cursos = df_cursos.drop(['Total geral','Total Geral','Total'],axis=1)
df_cursos = df_cursos.T

In [24]:
tags = {'História':['Historia','História Mosqueiro'],
'Artes Visuais': ['Artes visuais'],
'Terapia Ocupacional':['Terapia ocupacional'],
'Licenciatura Integrada em Educação em Ciências Matemática e Linguagens':['Licenciatura Integrada em Ciências Matemática e Linguagens','Licenciatura Integrada em educação'],
'Economia':['Ciências Econômicas'],
'Produção Multimídia':['Tecnologia em Produção Multimídia'],
'Arquitetura e Urbanismo':['Arquitetura e urbanismo'],
'Ciências Sociais':['Ciências Sociais Aplicadas'],
'Psicologia':['Intervalar Psicologia']
}

for tag in tags:
    can=df_cursos.loc[tag]
    for t in tags[tag]:
        can+=df_cursos.loc[t]
        df_cursos = df_cursos.drop(t)
    df_cursos.loc[tag]=can

In [25]:
df_cursos.sum()

2014    2272
2015    1150
2016     704
2017    1862
2018    4268
2019    1242
2020     731
2021     823
dtype: int64

In [26]:
for t in range(2014,2022):
    print('\n'.join([f'{x} - {y}' for (x,y) in zip(df_cursos[t].index,df_cursos[t])]))
    print('---------------------------------------------------------------')

Cinema e Audiovisual - 10
Dança - 23
Educação Artística - 2
Museologia - 24
Música - 27
Teatro - 20
Biomedicina - 15
Biotecnologia - 18
Ciências Biológicas - 54
Educação Física - 19
Pedagogia - 131
Ciência da Computação - 46
Ciências Naturais - 33
Estatística - 58
Física - 113
Matemática - 109
Química - 101
Química Industrial - 18
Sistemas de Informação - 18
Direito - 34
Enfermagem - 46
Farmácia - 91
Fisioterapia - 1
Medicina - 8
Nutrição - 37
Odontologia - 47
Administração - 48
Arquivologia - 2
Biblioteconomia - 21
Ciências Contábeis - 60
Serviço Social - 23
Turismo - 54
Licenciatura Integrada em Educação em Ciências Matemática e Linguagens - 12
Ciências Sociais - 78
Filosofia - 44
Geografia - 46
Psicologia - 32
Geofísica - 11
Geologia - 13
Meteorologia - 26
Oceanografia - 13
Comunicação Social - 22
Letras - 108
Engenharia Civil - 78
Engenharia da Computação - 44
Engenharia de Alimentos - 16
Engenharia de Telecomunicações - 2
Engenharia Elétrica - 65
Engenharia Mecânica - 100
Engenhar

In [27]:
def remove(a,b):
    global df_cursos
    

In [30]:
can=[]
for t in range(8):
    print(sum([int(x[-1]) for x in list_frames[t][list_frames[t].iloc[:,0].str.contains('Matemática')].values]))
    print('---------------------------------------------------')

109
---------------------------------------------------
32
---------------------------------------------------
23
---------------------------------------------------
109
---------------------------------------------------
161
---------------------------------------------------
45
---------------------------------------------------
34
---------------------------------------------------
29
---------------------------------------------------


In [29]:
df_cursos.to_csv(os.path.join(r'C:\Users\gabri\Workplace\LabEstat\code\data\out','datacursos.csv'), index=True)